In [ ]:
%run /Users/tyamgin/Projects/mlbootcamp/championship19/header.ipynb

In [3]:
class MyModel:
    verbose = 0
    def __init__(self, params={}):
        self.params = params
    def get_X(self, data):
        X = data.drop(['objectId', 'instanceId_userId'], 1)
        if 'label' in data.columns:
            X.drop('label', 1, inplace=True)
        return X
    def fit(self, data):
        pass
    def predict(self, X):
        return np.repeat(0.5, X.shape[0])
    #def return_and_cache():
    #hash(frozenset(my_dict.items()))
dummy_model = MyModel()
dummy_model.verbose

0

In [7]:
class MeanModel(MyModel):
    def __init__(self, models, coefs):
        self.models = models
        self.coefs = coefs
    def fit(self, data):
        for m in self.models:
            m.fit(data)
    def predict(self, X):
        sum = np.repeat(0.0, X.shape[0])
        for m, c in zip(self.models, self.coefs):
            sum += m.predict(X) * c
        return sum 

In [ ]:
def predict_to_submit(X, proba):
    X = X.assign(weight=-proba)
    scores = X[['instanceId_userId', 'objectId', 'weight']] \
            .groupby(['instanceId_userId','objectId']).mean()
    result = scores.sort_values(by=['instanceId_userId', 'weight']).reset_index()
    return result.groupby("instanceId_userId")['objectId'].apply(list)

In [37]:
def auc_true(test, pred):
    return mroc.mean_roc_auc(test.instanceId_userId.values, test.label.values, pred)


In [40]:
def cross_validation(model, data, n_folds=5, n_iters=5, seed=2707, verbose=1, gv={}):
    scores = []
    user_ids = data['instanceId_userId'].unique()
    for iter in range(n_iters):
        kf = KFold(n_splits=n_folds, random_state=seed+iter, shuffle=True)
        print(kf)
        fold = 0
        for train_index, test_index in kf.split(user_ids):
            fold += 1
            if verbose >= 2:
                print('Prepare data: %s' % datetime.datetime.now())
            data_train = data[data.instanceId_userId.isin(user_ids[train_index])]
            data_test = data[data.instanceId_userId.isin(user_ids[test_index])]
            gv['data_train'] = data_train
            gv['data_test'] = data_test

            if verbose >= 2:
                print('Fit: %s' % datetime.datetime.now())
            model.fit(data_train)
            if verbose >= 2:
                print('Predict: %s' % datetime.datetime.now())
            gv['model'] = model
            pred = model.predict(data_test.drop('label', 1))
            if verbose >= 2:
                print('Auc: %s' % datetime.datetime.now())
            score = auc_true(data_test, pred)
            scores.append(score)
            gv['pred'] = pred
            #if verbose >= 1:
            print('%2d -%2d : %1.4f, mean=%1.4f' % (iter, fold, score, np.mean(scores)))
    return np.mean(scores)

In [33]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [11, 12], [13, 14], [2, 3], [4, 5]])
y = np.array([0, 1, 1, 1, 0, 1, 0, 0])
data = pd.DataFrame(X, columns=['instanceId_userId', 'b'])
data['label'] = y
data

,instanceId_userId,b,label
0,1,2,0
1,3,4,1
2,1,2,1
3,3,4,1
4,11,12,0
5,13,14,1
6,2,3,0
7,4,5,0


In [34]:
#cross_validation(MyModel, data, n_folds=3, n_iters=1)

KFold(n_splits=3, random_state=2707, shuffle=True)
Prepare data: 2019-02-20 22:38:39.887536
Fit: 2019-02-20 22:38:39.893116
Predict: 2019-02-20 22:38:39.893417
Auc: 2019-02-20 22:38:39.894238
[array([11,  2]), array([0, 0]), array([0.5, 0.5])]
 0 - 1 : nan, mean=nan
Prepare data: 2019-02-20 22:38:39.896880
Fit: 2019-02-20 22:38:39.899935
Predict: 2019-02-20 22:38:39.900014
Auc: 2019-02-20 22:38:39.900826
[array([1, 3, 1, 3]), array([0, 1, 1, 1]), array([0.5, 0.5, 0.5, 0.5])]
 0 - 2 : 0.5000, mean=nan
Prepare data: 2019-02-20 22:38:39.902909
Fit: 2019-02-20 22:38:39.904595
Predict: 2019-02-20 22:38:39.904669
Auc: 2019-02-20 22:38:39.905394
[array([13,  4]), array([1, 0]), array([0.5, 0.5])]
 0 - 3 : nan, mean=nan


In [36]:
#cross_validation(MyModel, data, n_folds=3, n_iters=1)

KFold(n_splits=3, random_state=2707, shuffle=True)
Prepare data: 2019-02-20 22:38:55.203491
Fit: 2019-02-20 22:38:55.207606
Predict: 2019-02-20 22:38:55.207762
Auc: 2019-02-20 22:38:55.209564
[array([11,  2]), array([0, 0]), array([0.5, 0.5])]
 0 - 1 : nan, mean=nan
Prepare data: 2019-02-20 22:38:55.210607
Fit: 2019-02-20 22:38:55.213408
Predict: 2019-02-20 22:38:55.213497
Auc: 2019-02-20 22:38:55.215610
[array([1, 1, 3, 3]), array([0, 1, 1, 1]), array([0.5, 0.5, 0.5, 0.5])]
 0 - 2 : 0.5000, mean=nan
Prepare data: 2019-02-20 22:38:55.217106
Fit: 2019-02-20 22:38:55.220113
Predict: 2019-02-20 22:38:55.220198
Auc: 2019-02-20 22:38:55.221823
[array([13,  4]), array([1, 0]), array([0.5, 0.5])]
 0 - 3 : nan, mean=nan


In [20]:
_feature_selection_last_result = []
def feature_selection(train, model, start_features, required_columns):
    get_score = lambda feats: cross_validation(model, train[feats + required_columns], n_iters=1, verbose=1, seed=430)

    best_score = get_score(start_features)
    best_features = start_features
    print('Start with score=%.6f: %s' % (best_score, ','.join(best_features)))
    for i in range(len(start_features)):
        remove_feature = start_features[i]
        print('Try to remove %s' % remove_feature)
        try_features = [c for c in best_features if c != remove_feature]
        try_score = get_score(try_features)
        if try_score > best_score:
            best_score = try_score
            best_features = try_features
            global _feature_selection_last_result
            _feature_selection_last_result = best_features
            print('Found [%s] score=%.6f: %s' % (remove_feature, best_score, ','.join(best_features))) 


In [38]:
SnowballStemmer("russian").stem

<bound method RussianStemmer.stem of <RussianStemmer>>

In [39]:
1

1